In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

print(tf.__version__)

2.15.0


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Load original dataset and trained model

In [5]:
# loading dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [6]:
input_shape = (28, 28, 1)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test = x_test / 255.0

In [7]:
# one hot encoding
# y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [8]:
record_defaults = [tf.constant([], dtype=tf.float32)] * 10

y_train = tf.data.experimental.CsvDataset(
    filenames=["/content/drive/MyDrive/noisy_y_train.csv"], record_defaults=record_defaults
)

In [9]:
# Convert the CsvDataset to a list of EagerTensors
y_train = [tf.convert_to_tensor(value) for value in y_train]

In [10]:
len(y_train)

60000

Getting model and all original hyperparameters

In [11]:
batch_size = 64
num_classes = 10
num_epochs = 30

In [12]:
optimizer = tf.keras.optimizers.SGD(name="SGD")
cce = tf.keras.losses.CategoricalCrossentropy()

In [13]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(32,(5, 5), padding="same", activation="relu", input_shape=input_shape, name="conv1_1"),
        tf.keras.layers.Conv2D(32, (5, 5), padding="same", activation="relu", name="conv1_2"),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu", name="conv2_1"),
        tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu", name="conv2_2"),
        tf.keras.layers.MaxPool2D(strides=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu", name="FC_1"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation="relu", name="FC_2"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax", name="FC_softmax"),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.SGD(name="SGD"),
    loss="categorical_crossentropy",
    metrics=["acc"],
)

In [14]:
# Load gradients from the checkpoint file
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint.restore("/content/drive/MyDrive/epoch_layer_gradients.ckpt-1").expect_partial()

### Get noisy and clean samples

In [15]:
# get indices with noisy data
indices = (np.loadtxt("/content/drive/MyDrive/noisy_indices.csv", delimiter=",")).astype(int)
indices = indices.tolist()

# convert eagertensor to numpy
# y_train_numpy = [value.numpy() for value in y_train]

# create a x_batch of 1000 values using the 6k noisy examples
noisy_values_y = [y_train[i] for i in indices]
noisy_sample_y = noisy_values_y[:6000:6]

noisy_values_x = [x_train[i] for i in indices]
noisy_sample_x = noisy_values_x[:6000:6]

In [16]:
# get clean indices
total_indices = list(range(60000))
clean_indices = [value for value in total_indices if value not in indices]

# create a sample of 1000 values using the remaining 54k clean examples
clean_values_y = [y_train[i] for i in clean_indices]
clean_sample_y = clean_values_y[:54000:54]

clean_values_x = [x_train[i] for i in clean_indices]
clean_sample_x = clean_values_x[:54000:54]

In [17]:
noisy_sample_x = tf.convert_to_tensor(noisy_sample_x)
noisy_sample_y = tf.convert_to_tensor(noisy_sample_y)
clean_sample_x = tf.convert_to_tensor(clean_sample_x)
clean_sample_y = tf.convert_to_tensor(clean_sample_y)

In [18]:
print(noisy_sample_x[0].shape)
print(noisy_sample_y[0].shape)
noisy_sample = zip(noisy_sample_x, noisy_sample_y)
clean_sample = zip(clean_sample_x, clean_sample_y)

(28, 28, 1)
(10,)


The following summarizes, what we have to work with now:


*   clean_sample_x and clean_sample_y are 1000 samples of clean data
*   noisy_sample_x and noisy_sample_y are 1000 samples of noisy data



In [19]:
print("\t","noisy_sample","\t", "clean_sample", "\t","type")
print("x shape" , noisy_sample_x[0].shape,"\t",clean_sample_x[0].shape,"\t", type(noisy_sample_x[0]))
print("y shape" , noisy_sample_y[0].shape,"\t","\t",clean_sample_y[0].shape, "\t","\t",type(clean_sample_y[0]))
print("total", type(noisy_sample_x), len(noisy_sample_x))
print("total", type(clean_sample_x), len(clean_sample_x))
print("total", type(noisy_sample_y), len(noisy_sample_y))
print("total", type(clean_sample_y), len(clean_sample_y))

	 noisy_sample 	 clean_sample 	 type
x shape (28, 28, 1) 	 (28, 28, 1) 	 <class 'tensorflow.python.framework.ops.EagerTensor'>
y shape (10,) 	 	 (10,) 	 	 <class 'tensorflow.python.framework.ops.EagerTensor'>
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000
total <class 'tensorflow.python.framework.ops.EagerTensor'> 1000


### Load imp neurons for each example

In [20]:
import pickle

file_path = '/content/drive/MyDrive/examples_to_neurons_clean.pkl'
with open(file_path, 'rb') as file:
    example_to_neurons = pickle.load(file)
    

In [21]:

original_pred = model(clean_sample_x)
layers = [model.get_layer("conv1_1"), model.get_layer("conv1_2"), model.get_layer("conv2_1"), model.get_layer("conv2_2"), model.get_layer("FC_1"), model.get_layer("FC_2")]
# layer_weights has 6 layers. layer_weights[] gets the layer with kernel and bias both
# layer_weights = [layer.get_weights() for layer in layers]

flips_for_clean_example = []
for ex_idx , neuron_list in enumerate(example_to_neurons):
  # if ex_idx == 1:
  #   break

  # print("on example:", ex_idx)
  for layer_idx, neuron_idx in enumerate(neuron_list):
    # print("\t on layer:", layer_idx)

    layer = layers[layer_idx]
    original_weights = layer.get_weights()

    # Check if layer is Conv or FC layer and set both kernel and bias to 0
    if layer_idx in {0,1,2,3}:
      original_weights[0][:,:,:, neuron_idx] = 0.0
      original_weights[1][neuron_idx] = 0.0
    else:
      original_weights[0][:, neuron_idx] = 0.0
      original_weights[1][neuron_idx] = 0.0
    # print("check neuron no:", neuron_idx)
    # print(original_weights)
    layer.set_weights(original_weights)
    # make a pred on new model
    pred = model(tf.expand_dims(clean_sample_x[ex_idx], axis = 0))

    # check if prediction has flipped
    if tf.argmax(pred, axis = 1) != tf.argmax(original_pred[ex_idx]):
      flips_for_clean_example.append((ex_idx, layer_idx))
      break

  checkpoint.restore("/content/drive/MyDrive/epoch_layer_gradients.ckpt-1").expect_partial()


In [22]:
len(flips_for_clean_example)

35

In [23]:
checkpoint.restore("/content/drive/MyDrive/epoch_layer_gradients.ckpt-1").expect_partial()

In [24]:
file_path = '/content/drive/MyDrive/examples_to_neurons_noisy.pkl'
with open(file_path, 'rb') as file:
    example_to_neurons = pickle.load(file)

In [25]:
original_pred = model(noisy_sample_x)
layers = [model.get_layer("conv1_1"), model.get_layer("conv1_2"), model.get_layer("conv2_1"), model.get_layer("conv2_2"), model.get_layer("FC_1"), model.get_layer("FC_2")]


flips_for_noisy_example = []
for ex_idx , neuron_list in enumerate(example_to_neurons):
  # if ex_idx == 1:
  #   break

  # print("on example:", ex_idx)
  for layer_idx, neuron_idx in enumerate(neuron_list):
    # print("\t on layer:", layer_idx)

    layer = layers[layer_idx]
    original_weights = layer.get_weights()

    # Check if layer is Conv or FC layer and set both kernel and bias to 0
    if layer_idx in {0,1,2,3}:
      original_weights[0][:,:,:, neuron_idx] = 0.0
      original_weights[1][neuron_idx] = 0.0
    else:
      original_weights[0][:, neuron_idx] = 0.0
      original_weights[1][neuron_idx] = 0.0
    # print("check neuron no:", neuron_idx)
    # print(original_weights)
    layer.set_weights(original_weights)
    # make a pred on new model
    pred = model(tf.expand_dims(clean_sample_x[ex_idx], axis = 0))

    # check if prediction has flipped
    if tf.argmax(pred, axis = 1) != tf.argmax(original_pred[ex_idx]):
      flips_for_noisy_example.append((ex_idx, layer_idx))
      break

  checkpoint.restore("/content/drive/MyDrive/epoch_layer_gradients.ckpt-1").expect_partial()

In [26]:
len(flips_for_noisy_example)

896

In [27]:
flips_for_noisy_example


[(1, 0),
 (2, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (9, 0),
 (11, 0),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 0),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 0),
 (21, 0),
 (22, 0),
 (23, 0),
 (24, 0),
 (25, 0),
 (26, 0),
 (27, 0),
 (28, 0),
 (29, 0),
 (30, 0),
 (31, 0),
 (32, 0),
 (33, 0),
 (34, 0),
 (35, 0),
 (36, 0),
 (37, 0),
 (38, 0),
 (39, 0),
 (40, 0),
 (41, 0),
 (42, 0),
 (43, 0),
 (44, 0),
 (45, 0),
 (47, 0),
 (48, 0),
 (49, 0),
 (50, 0),
 (51, 0),
 (52, 0),
 (53, 0),
 (55, 0),
 (56, 0),
 (57, 0),
 (59, 0),
 (60, 0),
 (61, 0),
 (62, 0),
 (63, 0),
 (64, 0),
 (65, 0),
 (67, 0),
 (68, 0),
 (69, 0),
 (70, 0),
 (72, 0),
 (73, 0),
 (74, 0),
 (75, 0),
 (76, 0),
 (77, 0),
 (78, 0),
 (79, 0),
 (80, 0),
 (81, 0),
 (83, 0),
 (84, 0),
 (85, 0),
 (86, 0),
 (87, 0),
 (88, 0),
 (89, 0),
 (90, 0),
 (91, 0),
 (92, 0),
 (93, 0),
 (94, 0),
 (95, 0),
 (96, 0),
 (97, 0),
 (98, 0),
 (99, 0),
 (100, 0),
 (101, 0),
 (102, 0),
 (103, 0),
 (104, 0),
 (106, 0),
 (107, 0),
 (110, 0),
 (111, 0),
 (112, 0

In [28]:
flips_for_clean_example

[(5, 4),
 (42, 2),
 (43, 5),
 (58, 5),
 (91, 1),
 (131, 3),
 (141, 5),
 (354, 5),
 (414, 2),
 (422, 4),
 (474, 4),
 (503, 5),
 (550, 5),
 (557, 4),
 (581, 0),
 (615, 4),
 (631, 4),
 (687, 4),
 (693, 3),
 (761, 3),
 (768, 4),
 (779, 5),
 (791, 5),
 (793, 4),
 (798, 4),
 (799, 4),
 (805, 5),
 (817, 4),
 (830, 4),
 (897, 5),
 (898, 1),
 (949, 5),
 (958, 5),
 (963, 4),
 (990, 2)]